# Resume selector

Each resume is stored in the resumetext column. Resumes are reviewed for eligibility for data scientist positions. If a resume is selected, the applicant is invited for an interview.

### Install required libraries

In [1]:
!pip install nltk gensim wordcloud

#nltk: library for text preprocessing and linguistic analysis.
#Use cases: Tokenization, Stopword removal, Lemmatization/Stemming, Part-of-speech tagging, Named Entity Recognition

#Gensim: topic modeling and semantic similarity
#Use cases:
# - Building TF-IDF or word2vec models to capture context
# - Finding semantic similarity between resumes
# - Topic modeling with LDA (Latent Dirichlet Allocation)
# - Vectorizing text for machine learning models

#wordcloud: visualize the most frequent or important words in a corpus
# Use cases:
# - Quickly spotting dominant keywords in resumes
# - Creating visual summaries of skills, experiences
# - Exploratory data analysis (EDA) on text data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 12.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.metrics import classification_report, confusion_matrix

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

### update notebook style

In [2]:
!pip install jupyterthemes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.6 MB/s eta 0:00:00


In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

### load dataset

In [ ]:
resume_df = pd.read_csv('resume.csv', encoding = 'latin-1')
resume_df

In [ ]:
resume_df = resume_df[['resume_text','class']]

In [ ]:
resume_df.iloc[[0,-1],:]

### EDA

In [ ]:
resume_df.info()

In [ ]:
resume_df.isnull().sum()

In [ ]:
resume_df['class'].value_counts()

In [ ]:
resume_df['class'] = resume_df['class'].apply(lambda x:1 if x == 'flagged' else 0)
resume_df

In [ ]:
resume_df_0 = resume_df[resume_df['class']==0]
resume_df_1 = resume_df[resume_df['class']==1]
print('The number of 0 class',len(resume_df_0))
print('The number of 1 class',len(resume_df_1))

### Data cleaning

In [ ]:
resume_df['resume_text'] = resume_df['resume_text'].apply(lambda x:x.replace('\r', ''))
resume_df

In [ ]:
nltk.download('punkt')
nltk.download("stopwords")

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from','subject','reply','use','email','com'])

In [ ]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2 and token not in stop_words:
            result.append(token)

    return ' '.join(result)

In [ ]:
resume_df['cleaned'] = resume_df['resume_text'].apply(preprocess)

In [ ]:
print(resume_df['cleaned'][0])

In [ ]:
print(resume_df['resume_text'][0])

### Visualization

In [ ]:
sns.countplot(resume_df['class'], label = 'Count Plot')

In [ ]:
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 2000, width= 1600, height= 800, stopwords = stop_words).generate(str(resume_df[resume_df['class']==1].cleaned))
plt.imshow(wc)

In [ ]:
wc = WordCloud(max_words = 2000, width= 1600, height= 800, stopwords = stop_words).generate(str(resume_df[resume_df['class']==0].cleaned))
plt.imshow(wc)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
sample_data = ['This is the first document.','This document is the second document.','And this is the third one.','Is this the first document?']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sample_data)

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
print(X.toarray())

In [ ]:
vectorizer = CountVectorizer()
countvectorizer = vectorizer.fit_transform(resume_df['cleaned'])

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
print(countvectorizer.toarray())

### Train model

In [ ]:
X = countvectorizer

In [ ]:
y = resume_df['class']

In [ ]:
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

NB_classifier = MultinomialNB()
NB_classifier.fit(x_train, y_train)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.25)

In [ ]:
y_predict_train = NB_classifier.predict(x_train)
y_predict_train
cm = confusion_matrix(y_train, y_predict_train)
sns.heatmap(cm, annot = True)

In [ ]:
y_predict_test = NB_classifier.predict(x_test)
cm = confusion_matrix(y_test, y_predict_test)
sns.heatmap(cm, annot = True)

In [ ]:
print(classification_report(y_test, y_predict_test))